# Data Importing

This file includes the API calls to Alpha Vantage. We collect minute by minute data for 2022-2024. We had to make calls for one month at a time and aggregate into one csv file with all three years. Succesfully downloads data for over 1,400 stocks. 

We provided in the repository the 20 stocks' csv files (chosen by highest correlation as explained in report) that are in our final portfolio. **Running this file is not recomended**

In [1]:
import requests
import pandas as pd
import numpy as np 
import csv
import os
import time

In [ ]:
def fetch(tickers, api_key, months, folder_name):

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    # Fetch and save the data for each ticker
    call_count = 0
    for ticker in tickers:

        file_path = os.path.join(folder_name, f"{ticker}_2022-2024_1min.csv") # make name have timeframe
        

        if os.path.isfile(file_path):
            print(f"{ticker}_1min.csv already exists, check that it is correct.")
            continue
        else:
                url = "https://www.alphavantage.co/query"
                params = {
                    "function": "TIME_SERIES_INTRADAY",
                    "symbol": ticker,
                    "interval": "1min",
                    "month": months[0],
                    "outputsize": "full",
                    "apikey": api_key,
                    "extended_hours": "false"
                }
                # Fetch the data
                response = requests.get(url, params=params)
                call_count += 1
                try:
                    data = response.json()['Time Series (1min)']
                    main_df = pd.DataFrame.from_dict(data, orient='index')
                    main_df.dropna(inplace=True)
                    main_df = main_df.iloc[::-1]
                except Exception as e:
                    print(f"Could not get data for {ticker}")
                    print(response)
                    continue
                main_df = None
                for month in months[1:]:
                    url = "https://www.alphavantage.co/query"
                    params = {
                        "function": "TIME_SERIES_INTRADAY",
                        "symbol": ticker,
                        "interval": "1min",
                        "month": month,
                        "outputsize": "full",
                        "apikey": api_key,
                        "extended_hours": "false"
                    }
                    # Fetch the data
                    response = requests.get(url, params=params)
                    call_count += 1
                    try:
                        data = response.json()['Time Series (1min)']
                        df = pd.DataFrame.from_dict(data, orient='index')
                        df.dropna(inplace=True)
                        df = df.iloc[::-1]
                        main_df = pd.concat([main_df, df], join='inner')
                    except Exception as e:
                        print(f"Could not get data for {ticker}")
                        print(response)
                        continue
                if main_df is not None:
                    main_df.columns = ["open", "high", "low", "close", "volume"]
                    main_df.index.name = "timestamp"
                    main_df.to_csv(file_path, index=True)

                print(f"Data for {ticker} saved in {file_path}")
    return call_count

In [ ]:
# batch fetch used because API has 75 calls per minute limit

def batch_fetch(all_tickers, months, folder_name, api_key, batch_size=6, delay=60):
    api_call_count = 0
    for i in range(0, len(all_tickers), batch_size):
        batch = all_tickers[i:i+batch_size]
        api_call_count += fetch(batch, api_key, months, folder_name)
        if api_call_count >= 74:
            print("Waiting 1 minute before next batch...")
            time.sleep(delay)
            api_call_count = 0  # reset after delay


In [ ]:
tickers = pd.read_csv("full_tickers/2-1000_tickers.csv", header=None, names=['ticker'])

api_key = "R9ET6ZUJMPWT7N42"

start_date = pd.to_datetime("2025-02")


start_date = pd.to_datetime("2022-01")
end_date = pd.to_datetime("2024-12")

# Generate list of months between start and end dates
months = pd.date_range(start=start_date, end=end_date, freq='MS').strftime("%Y-%m").tolist()
folder_name = '2022-2024 1min data'

In [ ]:
batch_fetch(tickers["ticker"], months, folder_name, api_key, 6, 60)

LNT_1min.csv already exists, check that it is correct.
WRB_1min.csv already exists, check that it is correct.
NTR_1min.csv already exists, check that it is correct.
VRSK_1min.csv already exists, check that it is correct.
FUTU_1min.csv already exists, check that it is correct.
PRU_1min.csv already exists, check that it is correct.
WPM_1min.csv already exists, check that it is correct.
KMX_1min.csv already exists, check that it is correct.
STRL_1min.csv already exists, check that it is correct.
SNY_1min.csv already exists, check that it is correct.
FLEX_1min.csv already exists, check that it is correct.
ASTS_1min.csv already exists, check that it is correct.
APLD_1min.csv already exists, check that it is correct.
BIIB_1min.csv already exists, check that it is correct.
W_1min.csv already exists, check that it is correct.
SAIA_1min.csv already exists, check that it is correct.
APTV_1min.csv already exists, check that it is correct.
HES_1min.csv already exists, check that it is correct.
ATO